In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
import yaml
import logging
from datetime import datetime

# YAML config
try:
    with open(r".\config.yaml", "r") as f:
        config = yaml.safe_load(f)
except Exception as e:
    raise

# Logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(name)s - %(funcName)s - %(message)s",
    filename=config["log_dir"] +
    f"{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.log",
    filemode="w"
)
logger = logging.getLogger(__name__)

logger.info("Config file and logger setup completed.")

In [ ]:
# Data Loading & Preprocessing
def load_and_preprocess_data(images_path, size):
    """Loads, resizes, and normalizes Pokemon images."""
    try:
        images = []
        for f in os.listdir(images_path):
            f = os.path.join(images_path, f)
            try:
                img = Image.open(f).resize(size)
                img_array = np.array(img).astype(
                    "float32") / 255.0  # Normalize to [0, 1]
                images.append(img_array)
            except Exception as e:
                logger.warning(f"Error loading or processing image {f}: {e}")

        return np.array(images)

    except Exception as e:
        logger.error(f"Error loading images: {e}")
        return None

In [ ]:
images = load_and_preprocess_data(config["data_path"], config["size"])
images_count = len(images)

X_train = images[:int(images_count*0.8)]
X_test = images[int(images_count*0.8):]

N, H, W = X_train.shape[:-1]
D = H * W

In [ ]:
# Flattening the data
X_train = X_train.reshape(-1, D)
X_test = X_test.reshape(-1, D)

In [ ]:
# Models Definitions
def build_generator(latent_dim):
    try:
        i = Input(shape=(latent_dim,))
        x = Dense(256, activation=LeakyReLU(negative_slope=0.2))(i)
        x = BatchNormalization(momentum=0.8)(x)
        x = Dense(512, activation=LeakyReLU(negative_slope=0.2))(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Dense(1024, activation=LeakyReLU(negative_slope=0.2))(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Dense(D, activation="tanh")(x)  # -1 and 1 values used

        model = Model(i, x)
        return model
    except Exception as e:
        logger.error(f"Generator failed: {e}")
        return None


def build_discriminator(img_size):
    try:
        """Builds the discriminator model."""
        i = Input(shape=(img_size,))
        x = Dense(512, activation=LeakyReLU(negative_slope=0.2))(i)
        x = Dense(256, activation=LeakyReLU(negative_slope=0.2))(x)
        x = Dense(1, activation="sigmoid")(x)

        model = Model(i, x)
        model.compile(loss="binary_crossentropy", optimizer=Adam(
            learning_rate=0.0002, beta_1=0.5, beta_2=0.99, epsilon=1e-8), metrics=["accuracy"])

        return model
    except Exception as e:
        logger.error(f"Discriminator failed: {e}")
        return None

In [ ]:
discriminator = build_discriminator(D)
generator = build_generator(config["latent_dimansinality"])

In [ ]:
def combine_models(generator, discriminator):
    try:
        """Combines the generator and discriminator."""
        # Create an input to represent noise sample from latent space
        z = Input(shape=(config["latent_dimansinality"],))

        # Pass noise to get image
        img = generator(z)

        # Making sure only generator is trainable
        discriminator.trainable = False

        # True output is fake yet we label as real
        fake_pred = discriminator(img)

        combined_model = Model(z, fake_pred)
        combined_model.compile(loss="binary_crossentropy", optimizer=Adam(
            learning_rate=0.0002, beta_1=0.5, beta_2=0.99, epsilon=1e-8))

        return combined_model
    except Exception as e:
        logger.error(f"Combining models failed: {e}")

In [ ]:
combined_model = combine_models(generator, discriminator)

In [ ]:
# Utility Functions
def sample_image(epochs):
    """Generates and saves sample images at specified epochs."""
    generator = build_generator(config["latent_dimansinality"])

    rows, cols = 5, 5
    noise = np.random.randn(rows * cols, config["latent_dimansinality"])
    imgs = generator.predict(noise)
    imgs = 0.5 * imgs + 0.5
    fig, axs = plt.subplots(rows, cols)
    idx = 0
    for i in range(rows):
        for j in range(cols):
            axs[i, j].imshow(imgs[idx].reshape(H, W), cmap="gray")
            axs[i, j].axis("off")
            idx += 1
    fig.savefig(r".\%d.png" % epochs)
    plt.close()

In [ ]:
def validate_gan(generator, discriminator, X_train):
    """Validates the GAN by generating and displaying samples."""
    # Generate a batch of samples
    noise = np.random.randn(
        config["batch_size"], config["latent_dimansinality"])
    generated_images = generator.predict(noise)

    # Display the generated images
    fig, axes = plt.subplots(
        config["batch_size"], 1, figsize=(8, 4 * config["batch_size"]))
    for i in range(config["batch_size"]):
        axes[i].imshow(generated_images[i].reshape(H, W), cmap="gray")
        axes[i].axis("off")
    plt.show()

In [ ]:
# Training Loop
def train_gan(discriminator, combined_model, X_train, epochs, batch_size):
    """Trains the GAN."""
    try:
        ones = np.ones(batch_size)
        zeros = np.zeros(batch_size)

        d_losses = []
        g_losses = []

        for epoch in range(epochs):
            # Real images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_imgs = X_train[idx]

            # Generate fake images
            noise = np.random.randn(batch_size, config["latent_dimansinality"])
            fake_images = generator.predict(noise)

            # Train discriminator
            d_loss_real, d_acc_real = discriminator.train_on_batch(
                real_imgs, ones)
            d_loss_fake, d_acc_fake = discriminator.train_on_batch(
                fake_images, zeros)
            d_loss = 0.5 * (d_loss_real + d_loss_fake)

            # Train generator
            noise = np.random.randn(batch_size, config["latent_dimansinality"])
            g_loss = combined_model.train_on_batch(noise, ones)

            # Validation
            # if epoch % validation_frequency == 0:
            #     validate_gan(generator, discriminator, X_train) # Calls validation function

            if epoch % 100 == 0:
                print(
                    f"Epoch {epoch}, D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}")
                logger.info(
                    f"Epoch {epoch}, D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}")

            if epoch % config["sample_period"] == 0:
                sample_image(epoch)
    except Exception as e:
        logger.error(f"Training failed: {e}")

In [ ]:
train_gan(discriminator, combined_model, X_train,
          config["epochs"], config["batch_size"])